### ```Fichier initialement prévu pour la prédiction avec les modèles que l'on a entraîné et enregistré, n'a pas fonctionné pour une raison que j'ignore, je me suis donc rabattu sur la prédiction directement après entraînement, tant que le modèle était en mémoire```

In [21]:
import torch
import pandas as pd
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
from ResNet import *

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = f"{self.img_dir}/{self.img_labels.iloc[idx, 0]}.jpg"
        image = Image.open(img_path)
        #image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1] - 1
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

# Ajout de transformations sur les images
data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.CenterCrop(256),
    # DATA AUGMENTATION
    transforms.RandomRotation(degrees=90),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    # FIN DATA AUGMENTATION
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5)
])

test_images = CustomImageDataset(img_dir='test/test', transform=data_transforms)
test_loader = DataLoader(test_images, batch_size=32, shuffle=False, num_workers=4)

modelload = ResNet()
poids = torch.load('ResNet18_60_epochs.pth')

#del poids['fc.weight']
#del poids['fc.bias']

modelload.load_state_dict(poids)

# On met le modèle en mode prédiction
modelload.eval()

predictions = []

with torch.no_grad():
    for batch_idx, (images, filenames) in enumerate(test_loader):
        outputs = modelload(images)
        _, predicted = torch.max(outputs, 1)
        # Ajoutez les prédictions converties de 0-3 à 1-4 dans la liste des prédictions
        predictions.extend(predicted.cpu().numpy() + 1)

# Export vers CSV
file_ids = [filename.split('.')[0] for filename in filenames]
df = pd.DataFrame({'id': file_ids, 'label': predictions})
df.to_csv('predictions.csv', index=False)


RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([1000, 512]) from checkpoint, the shape in current model is torch.Size([4, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([4]).